In [213]:
import pandas as pd

mini = pd.read_csv('def/phi-mini/cot/phi-mini-cot.csv')
med = pd.read_csv('def/phi-medium/cot/phi-medium-cot.csv')
two = pd.read_csv('def/gemma-2-2b-it/cot/gemma-2-2b-it-cot.csv')
nine = pd.read_csv('def/gemma-2-9b-it/cot/gemma-2-9b-it-cot.csv')

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from guidance import models, select

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [231]:
two.head()

,Unnamed: 0,query,correct,thesis,antithesis,pre-synthesis,synthesis,context
0,0,"Which magazine was started first, Arthur's Mag...",arthurs magazine,arthurs magazine,"Assistant: \n\nThe context states that ""First ...",The correct answer is **Arthur's Magazine**. \...,arthurs magazine,Arthur's Magazine (1844–1846) was an American ...
1,1,Which tennis player won more Grand Slam titles...,jonathan stark,henri leconte,Assistant: \n\nThe context provides informatio...,The most correct answer is **Henri Leconte**. ...,henri leconte,Henri Leconte (born 4 July 1963) is a former F...
2,2,"Which band was founded first, Hole (the rock b...",the wolfhounds,the wolfhounds,Assistant: \n\nThe context states that Hole wa...,The correct answer is **The Wolfhounds**. \n\n...,the wolfhounds,The Wolfhounds are an indie pop/noise pop band...
3,3,Were Pavel Urysohn and Leonid Levin known for ...,no,no,Assistant: \n\nThe context focuses on the work...,The most correct answer is **no**. \n\nThe con...,yes,Leonid Anatolievich Levin ( ; Russian: Леони́д...
4,4,Are both The New Pornographers and Kings of Le...,yes,yes,Assistant: \n\nLet's analyze the options and t...,The correct answer is **no**. \n\nHere's why:\...,yes,Kings of Leon is an American rock band that fo...


In [232]:
two = two.drop(['Unnamed: 0'], axis = 1)

In [4]:
from langchain_core.prompts import PromptTemplate
from operator import itemgetter

# prompt augmentation for the (format of the) synthesis:
prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
Choose the most proper option between {options} that best matches with the suggestion. 

Question: {question}
Context: {critique}
Sources: {context}

Assistant:
"""
)
augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }
synthesis_chain = augmentation | prompt_template 

In [5]:
import ast
import re

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

def synthesisGeneration(query, merged, pre_answer, sources):
    merged = ast.literal_eval(merged)
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'critique': pre_answer,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select(merged)
    return str(ans)

def extract_answer_synthesis(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("\n\nAssistant:\n")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("\n\nAssistant:\n")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", use_fast=False)
new_model = models.Transformers(model, tokenizer, temperature=0.0)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/guidance/chat.py:73: UserWarning: Chat template {% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %} was unable to be loaded directly into guidance.
                        Defaulting to the ChatML format which may not be optimal for the selected model. 
                        For best results, create and pass in a `guidance.ChatTemplate` subclass for your model.
  warnings.warn(f"""Chat template {chat_template} was unable to be loaded directly into guidance.


In [7]:
import datasets
from datasets import load_dataset

dataset = load_dataset('saracandu/hotpotQA_nli', split="train", trust_remote_code=True)

# select a subset of the queries, just for test:
first_queries = dataset['question']

# same for correct answers and distractors:
correct_answers = dataset['answer']
possibilities = dataset['options']

# and for the sources:
sources = dataset['passages']

In [239]:
syn_answers = []
for i in range(len(dataset)):
    syn_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            nine['pre-synthesis'][i], sources[i])))

In [240]:
ant_answers = []
for i in range(len(dataset)):
    ant_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            nine['antithesis'][i], sources[i])))

In [208]:
answers = []
for i in range(len(dataset)):
    answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            nine['cot'][i], sources[i])))

In [241]:
df = {
    'query': first_queries,
    'correct': correct_answers,
    'thesis': nine['thesis'],
    'pre-antithesis': nine['antithesis'],
    'antithesis': ant_answers,
    'pre-synthesis': nine['pre-synthesis'],
    'synthesis': syn_answers,
    'context': sources
} 

df = pd.DataFrame(df)

In [209]:
df = {
    'query': first_queries,
    'correct': correct_answers,
    'purecot': nine['cot'],
    'answer': answers,
    'context': sources
} 

df = pd.DataFrame(df)

In [219]:
def clean_text_final(text):
    text = re.sub(r'[^\w\s.,!?\'"\-:;()]+', '', text)  # Rimuove i caratteri speciali
    text = re.sub(r"['\"-]", '', text)  # Rimuove apostrofi, virgolette e trattini
    text = text.lower()  # Converte in minuscolo
    return text

In [242]:
# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text_final)
df['thesis'] = df['thesis'].apply(clean_text_final)
df['antithesis'] = df['antithesis'].apply(clean_text_final)
df['synthesis'] = df['synthesis'].apply(clean_text_final)

In [210]:
# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text_final)
df['answer'] = df['answer'].apply(clean_text_final)

In [243]:
df.head()

,query,correct,thesis,pre-antithesis,antithesis,pre-synthesis,synthesis,context
0,"Which magazine was started first, Arthur's Mag...",arthurs magazine,arthurs magazine,Assistant: \n\nThe context states that Arthur'...,arthurs magazine,You are absolutely correct! \n\nHere's a break...,arthurs magazine,Arthur's Magazine (1844–1846) was an American ...
1,Which tennis player won more Grand Slam titles...,jonathan stark,henri leconte,Let's analyze the context:\n\n* **Henri Lecont...,jonathan stark,You are absolutely right! \n\nThe candidate an...,jonathan stark,Henri Leconte (born 4 July 1963) is a former F...
2,"Which band was founded first, Hole (the rock b...",the wolfhounds,the wolfhounds,Assistant: \n\nThe context states that Hole wa...,the wolfhounds,You are absolutely right!\n\nThe context clear...,the wolfhounds,The Wolfhounds are an indie pop/noise pop band...
3,Were Pavel Urysohn and Leonid Levin known for ...,no,yes,Assistant: \n\nLet's analyze the context and t...,no,You've done a great job analyzing the context!...,no,Leonid Anatolievich Levin ( ; Russian: Леони́д...
4,Are both The New Pornographers and Kings of Le...,yes,yes,Assistant: \n\nLet's analyze the context:\n\n*...,no,You are absolutely right! \n\nThe context clea...,no,Kings of Leon is an American rock band that fo...


In [244]:
df.to_csv('cot-gemma-9b-hotpotqa.csv')